In [ ]:
#content_filtering code

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_excel("sample_data.xlsx")

# Preprocessing
# Encode categorical variables
label_encoders = {}
for col in ["state", "category"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split data
X = data.drop(["place1", "place2", "place3"], axis=1)
y = data[["place1", "place2", "place3"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model (example using RandomForestClassifier)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# User input (state, category, rating)
user_input_state = "Uttarakhand"  # Replace with actual user input
user_input_category = "Religious"
user_input_rating = 4.5

# Process user input
user_input_encoded = pd.DataFrame({
    "state": [label_encoders["state"].transform([user_input_state])[0]],
    "category": [label_encoders["category"].transform([user_input_category])[0]],
    "rating": [user_input_rating]
})

# Get recommendations
predictions = model.predict(user_input_encoded)

print("Top 3 recommended places:", predictions)


Top 3 recommended places: [['Haridwar' ' Panch Kedar' ' Hemkund Sahib']]


In [ ]:
#Content_filtering_using_tensorflow

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

# Output layers for place1, place2, and place3
output1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer)
output2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer)
output3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Hill Station'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')



Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - dense_77_accuracy: 0.0096 - dense_78_accuracy: 0.0298 - dense_79_accuracy: 0.0256 - loss: 13.4359 - val_dense_77_accuracy: 0.1136 - val_dense_78_accuracy: 0.0767 - val_dense_79_accuracy: 0.1364 - val_loss: 12.5596
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_77_accuracy: 0.0978 - dense_78_accuracy: 0.0672 - dense_79_accuracy: 0.1109 - loss: 12.4705 - val_dense_77_accuracy: 0.1420 - val_dense_78_accuracy: 0.0938 - val_dense_79_accuracy: 0.1619 - val_loss: 11.2408
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_77_accuracy: 0.0975 - dense_78_accuracy: 0.0662 - dense_79_accuracy: 0.1213 - loss: 11.4205 - val_dense_77_accuracy: 0.1875 - val_dense_78_accuracy: 0.1335 - val_dense_79_accuracy: 0.1932 - val_loss: 9.8574
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_77_accuracy: 0.1649 - dense_78_accuracy: 0.0892 - dense_79_accuracy: 0.1648 - loss: 10.1836 - val_dense_77_accuracy: 0.2585 - val_dense_78_a

C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Recommended places: ('Haldwani', ' Pauri', ' Bhowali')


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

# Output layers for place1, place2, and place3
output1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer)
output2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer)
output3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Save the model with a .keras extension
model.save('my_model.keras')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model('my_model.keras')
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Hill Station'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')


Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - dense_12_accuracy: 0.0181 - dense_13_accuracy: 0.0162 - dense_14_accuracy: 0.0338 - loss: 13.3998 - val_dense_12_accuracy: 0.1136 - val_dense_13_accuracy: 0.0426 - val_dense_14_accuracy: 0.0767 - val_loss: 12.4856
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_12_accuracy: 0.0897 - dense_13_accuracy: 0.0423 - dense_14_accuracy: 0.0678 - loss: 12.4602 - val_dense_12_accuracy: 0.1705 - val_dense_13_accuracy: 0.1108 - val_dense_14_accuracy: 0.1364 - val_loss: 11.5423
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_12_accuracy: 0.1396 - dense_13_accuracy: 0.0841 - dense_14_accuracy: 0.1000 - loss: 11.5731 - val_dense_12_accuracy: 0.1875 - val_dense_13_accuracy: 0.1364 - val_dense_14_accuracy: 0.1648 - val_loss: 10.3799
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - dense_12_accuracy: 0.1632 - dense_13_accuracy: 0.1074 - dense_14_accuracy: 0.1277 - loss: 10.6544 - val_dense_12_accuracy: 0.2159 - val_dense_13_

AttributeError: 'str' object has no attribute 'call'

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

# Output layers for place1, place2, and place3
output1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer)
output2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer)
output3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Save the model with .h5 extension
model.save('my_model.h5')

# Load the model back from the .h5 file
loaded_model = tf.keras.models.load_model('my_model.h5')

# Define the input signature
input_signature = [tf.TensorSpec(shape=(None, input_shape), dtype=tf.float32)]

# Create a concrete function from the Keras model
concrete_func = tf.function(lambda x: loaded_model(x))
concrete_func = concrete_func.get_concrete_function(input_signature)

# Convert the loaded model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = loaded_model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Hill Station'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')


Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - dense_7_accuracy: 0.0213 - dense_8_accuracy: 0.0161 - dense_9_accuracy: 0.0125 - loss: 13.4235 - val_dense_7_accuracy: 0.1562 - val_dense_8_accuracy: 0.0994 - val_dense_9_accuracy: 0.1364 - val_loss: 12.4323
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_7_accuracy: 0.1719 - dense_8_accuracy: 0.0631 - dense_9_accuracy: 0.1076 - loss: 12.2017 - val_dense_7_accuracy: 0.1534 - val_dense_8_accuracy: 0.1278 - val_dense_9_accuracy: 0.1619 - val_loss: 11.0424
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_7_accuracy: 0.1267 - dense_8_accuracy: 0.0853 - dense_9_accuracy: 0.1204 - loss: 11.0912 - val_dense_7_accuracy: 0.1847 - val_dense_8_accuracy: 0.1335 - val_dense_9_accuracy: 0.1648 - val_loss: 9.6821
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_7_accuracy: 0.1819 - dense_8_accuracy: 0.0965 - dense_9_accuracy: 0.1535 - loss: 9.9882 - val_dense_7_accuracy: 0.2131 - val_dense_8_accuracy: 0.1506 - val_de

C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Recommended places: ('Haldwani', ' Roorkee', ' Bhowali')


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

print (y1)

[33 32 16 ... 15 68  3]


In [9]:
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    print( state_encoded)
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = loaded_model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Hill Station'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')


C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[1]


NameError: name 'loaded_model' is not defined